## Energy Scan
In this example we will compute and fit a potenital energy curve for the Helium dimer.

In [ ]:
import psi4

To begin with lets create a string representation of our Helium dimer where the `--` seperates the two Helium atoms into fragments (important later) and the variable `**R**` represents the distance between the Helium atoms.

In [ ]:
mol_string = """
He
--
He 1 **R**
"""

Now we can build a series of Helium atoms at different seperations and compute the energy at each points.

In [ ]:
distances = [2.875, 3.0, 3.125, 3.25, 3.375, 3.5, 3.75, 4.0, 4.5, 5.0, 6.0, 7.0]
energies = []
for d in distances:
    # Build a new molecule at each seperation
    mol = psi4.geometry(mol_string.replace('**R**', str(d)))
    
    # Compute the Countpoise-Corrected interaction energy
    en = psi4.energy('MP2/aug-cc-pVDZ', molecule=mol, bsse_type='cp')

    # Place in a reasonable unit, Wavenumbers in this case
    en *= psi4.p4const.psi_hartree2wavenumbers
    
    # Append the value to our list
    energies.append(en)

print("Finished computing the potential!")

We can use the `NumPy` library to fit a curve. In this case we will fit a Lennard-Jones potential.

In [ ]:
import numpy as np

In [ ]:
# Fit the data in a linear leastsq way to a -12, -6 polynomial
powers = [-12, -6]
x = np.power(np.array(distances).reshape(-1, 1), powers)
coefs = np.linalg.lstsq(x, energies)[0]

# Build a list of points
fpoints = np.linspace(2, 7, 50).reshape(-1, 1)
fdata = np.power(fpoints, powers)

fit_energies = np.dot(fdata, coefs)

To visualize our results we can use the matplotlib library. The `%matplotlib inline` is a "magic" iPython command so that the graphs will show up in the notebook itself.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.xlim((2, 7)) # X limits
plt.ylim((-7, 2)) # Y limits
plt.scatter(distances, energies) # Scatter plot of the distances/energies
plt.plot(fpoints, fit_energies) # Fit data
plt.plot([0,10], [0,0], 'k-') # Make a line at 0